In [1]:
import networkx as nx
import pickle as pkl
from pyvis import network as net
from IPython.display import Image
import numpy as np
import functools
import pwb
import pywikibot
import wptools
import re

# Analysis of Wikipedia information

Two sources of semantic structured knowledge: infoboxes and categories.

## Infoboxes

"Infobox templates contain important facts and statistics of a type which are common to related articles". There are pre-defined infobox templates for many "categories" https://en.wikipedia.org/wiki/Wikipedia:List_of_infoboxes:

* https://en.wikipedia.org/wiki/Template:Infobox_poker_player
* https://en.wikipedia.org/wiki/Template:Infobox_writer
* https://en.wikipedia.org/wiki/Template:Infobox_officeholder

An infobox can be seen as a RDF graph of subject:predicate:object triples.

The predicates are similar for similar pages e.g. politicians, writers, and they can be used to identify a "general" type of an entity.

Example for two politicians (property "Prime Minister" shared suggests that both have worked as politicians):

        Nicolas Sarkozy : office : Minister of the Interior
        Nicolas Sarkozy : Prime Minister : 	Dominique de Villepin

        François Bayrou : office : Minister of Justice
        François Bayrou : Prime Minister : 	Édouard Philippe

Example for two countries with shared "capital" predicate:

        France : capital : Paris
        Italy : capital : Rome


The objects identify more specific features of the entities:

Example for two politicians, the value specifies the political party and even the "procedence", useful for inferring e.g. French Politicians

        François Bayrou : Political party : Union for French Democracy

        Nicolas Sarkozy : Political party : The Republicans (France)

Example for two countries, ideally, the euro could be useful for inferring e.g. european countries and the coordinates could be useful for detecting the cardinal points of the location.

        Italy : Currency : Euro (€)b (EUR)

        France : Currency : Euro (€)b (EUR)

        Italy : Coordinates : 41°54′N 12°29′E

        France : Coordinates : 48°51′N 2°21′E

Example for two corporations (computer companies):

        Google : Industry : Computer software ...

        Microsoft : Industry : Computer software ...


https://pypi.org/project/wptools/0.2.1/ as parser for extracting the infoboxes (there aren't parsers of infoboxes
for MediaWiki):

        'conventional_long_name': 'French Republic',
        'common_name': 'France',
        'native_name': '{{native name|fr|République française|nbsp|=|omit}}',
        'national_anthem': "[[La Marseillaise]]",
        'capital': '[[Paris]]',
        'coordinates': '{{Coord|48|51|N|2|21|E|type:city}}',
        ...

## Categories

I have written code to get a graph of Wikipedia categories (a script for ***pywikibot [Wikipedia foundation]***). It is a DFS with depth 6 from the categories in the Wikipedia page of each entity in TESA. The search finishes if the maximum depth is exceeded or roots are reached ('culture', 'economy', 'education', ..., https://en.wikipedia.org/wiki/Category:Main_topic_classifications) or an artificial category is reached ("wikipedia", "hidden", "category", ...). There is an edge between u and v if v is a sub-category of u in Wikipedia. It recovers 73021 nodes and 209490 edges (similar to https://wikiworkshop.org/2019/papers/Wiki_Workshop_2019_paper_9.pdf). 

But the structure of categories in Wikipedia is not a hierarchy, there are cycles generated by human-artifacts that connect sub-categories to categories (Figure 2). So, we can't do things like computing the lowest common ancestor among entities' categories. Also, a category can have multiple parents (Figure 1) and there are multiple roots.

It's a graph of categories that relates similar categories but not exactly a hierarchy as a whole.

Also, large walks in the graph could relate categories in a "stranger" way as shown in Figure 3 (limiting the length of the walks alleviate the problem).

In [2]:
with open("./knowledge_graphs/category_graph_depth-6_dag.pkl", "rb") as fr:
    dag = pkl.load(fr)
    
with open("./knowledge_graphs/category_graph_depth-6.pkl", "rb") as fr:
    graph = pkl.load(fr)

#graph = graph.reverse()

subgraph = graph.subgraph(["English politicians"] + list(graph.successors("English politicians")))

print("Example of multiple parents")
g = net.Network(notebook=True, height="500px", width="900px", directed=True)
g.show_buttons(filter_=['physics'])
g.from_nx(subgraph)
g.show("_.html")

Example of multiple parents


In [3]:
gen = nx.simple_cycles(graph)
for i in range(110):
    cycle = next(gen)
#cycle = next(gen)
cycle_subgraph = graph.subgraph(cycle) #["Bleeding Kansas", "Kansas Territory"]) #cycle)

print("Example of cycle that starts and ends in %s (last category: %s)" % (cycle[0], cycle[-1]))

g = net.Network(notebook=True, height="500px", width="900px", directed=True)
g.show_buttons(filter_=['physics'])
g.from_nx(cycle_subgraph)
g.show("_.html")

Example of cycle that starts and ends in Rivers of Kern County, California (last category: San Joaquin River)


In [4]:
key1 = "Taiwan"
key2 = "XML"

print("How to relate %s with %s:\n" % (key1, key2))

g = net.Network(notebook=True, height="500px", width="900px", directed=True)
g.show_buttons(filter_=['physics'])
g.from_nx(graph.subgraph(next(nx.all_shortest_paths(graph, key1, key2))))
g.show("_.html")

How to relate Taiwan with XML:



### Breaking Cycles

Few "recent" works in the literature tried to generate a DAG from noisy knowledge graphs with cycles:

* Efficient Pruning of Large Knowledge Graphs (https://www.ijcai.org/Proceedings/2018/0564.pdf):
Given a (directed) noisy knowledge graph NKG(V;E)(hereafter denoted G for brevity), a set P \subset V of terminological nodes and crumbs to be preserved (referred to  as protected nodes),  and  a  root r \in P,  CRUMBTRAIL prunes G to obtain an acyclic subgraph G_P that contains all nodes of P, as well as possibly other nodes to guarantee connectivity properties as explained below...



* Breaking Cycles in Noisy Hierarchies (https://par.nsf.gov/servlets/purl/10028167): Remove cycles while preserving the logical structure (hierarchy) of a directed graph as much as possible. Consider a ranking function f that assigns a ranking score to each node in the graph. A higher ranking score implies that the corresponding node is higher up (e.g., more general) in the hierarchy. Given such a ranking, the edges which violate the hierarchy (i.e., edges from a higher/general group to a lower/specic group) are potential candidates for removal. Thus in our approach, there are two sub-tasks involved:

    * Inferring graph hierarchy (finding a ranking function)
    * Proposing strategies to select violation edges as candidates for removal
    
Inferring graph hierarchy: The best performing and simple system is based in TrueSkill. It is a Bayesian skill rating system designed to calculate the relative skill of players from the set of generated competitions in multi-player games - Xbox Live rankings - . They transform a directed graph G=(V,E) into a multiplayer tournament with |V| players and |E| competitions. For each edge (u,v) \in E, we consider that u loses the competition between u and v. Based on the current estimated skill levels of two players (u and v) and the outcome of a new game between them (edge(u,v)), the TrueSkill model updates the skill level μ and σ intuitively based on whether the outcome of the new competition is expected or unexpected.

Selecting violation edges: Greedy, remove the edges that violates the hierarchy the most.

I adapted the code of Breaking Cycles in Noisy Hierarchies (https://github.com/zhenv5/breaking_cycles_in_noisy_hierarchies) for python 3 and networkx 2.4 for creating a DAG of the categories.

Are there cycles?

In [5]:
print(len(list(nx.simple_cycles(dag)))>0)

False


Now, we can compute things like lowest common ancestors:

In [6]:
def lowest_common_ancestors(graph, nodes, top_k):
    assert nx.is_directed_acyclic_graph(graph), "Graph has to be acyclic and directed."

    common_ancestors = list(set.intersection(*[nx.ancestors(graph, node) for node in nodes if node != "Living people" or node != "People"]))

    if not common_ancestors:
        return []
    
    sum_of_path_lengths = np.zeros((len(common_ancestors)))
    for ii, c in enumerate(common_ancestors):
        sum_of_path_lengths[ii] = functools.reduce(lambda x, y: x + y, 
                                                   [nx.shortest_path_length(graph, c, node) 
                                                         for node in nodes])

    indices = np.argsort(sum_of_path_lengths)[:top_k]   
    return [common_ancestors[ii] for ii in indices], sum_of_path_lengths[indices]

In [7]:
def highest_common_descendants(graph, nodes, top_k):
    assert nx.is_directed_acyclic_graph(graph), "Graph has to be acyclic and directed."

    common_descendants = list(set.intersection(*[nx.descendants(graph, node) for node in nodes]))

    if not common_descendants:
        return []
    
    sum_of_path_lengths = np.zeros((len(common_descendants)))
    for ii, c in enumerate(common_descendants):
        sum_of_path_lengths[ii] = functools.reduce(lambda x, y: x + y, 
                                                   [nx.shortest_path_length(graph, node, c) 
                                                         for node in nodes])

    indices = np.argsort(sum_of_path_lengths)[:top_k]   
    return [common_descendants[ii] for ii in indices], sum_of_path_lengths[indices]

In [8]:
# Some entities are also categories: Barack Obama, Donald Trump and Boeing
keys1 = ["Cities in North America", "Barack Obama", "French politicians", "Boeing"]
keys2 = ["Capitals", "Donald Trump", "American politicians", "Aerospace companies of Europe"]
top_k = 3

print("Lower common ancestors:")
for k1, k2 in zip(keys1, keys2):
    print("\n(%s, %s):" % (k1, k2), lowest_common_ancestors(dag, [k1, k2], top_k=top_k), "\n")

Lower common ancestors:

(Cities in North America, Capitals): (['Cities', 'Municipalities', 'City'], array([2., 4., 4.])) 


(Barack Obama, Donald Trump): (['Living people', 'Presidents of the United States', 'American male non-fiction writers'], array([2., 2., 2.])) 


(French politicians, American politicians): (['Politicians', 'Political parties', 'Causes of death'], array([2., 4., 4.])) 


(Boeing, Aerospace companies of Europe): (['Aerospace companies', 'Transport companies', 'Spaceflight'], array([3., 5., 5.])) 



Some of previous strange relationships do not appear in the DAG:

In [66]:
keys1 = ["Taiwan", "Stalinism", "Entertainers", "Politicians"]
keys2 = ["XML", "Socialism", "1940s in Japan", "French politicians"]


for k1, k2 in zip(keys1, keys2):
    print("(%s, %s)\n" % (k1, k2))
    try:
        nx.shortest_path(graph, k1, k2)
        print("(RAW) There is path")
    except:
        print("(RAW) There isn't path")

    try:
        nx.shortest_path(dag, k1, k2)
        print("(DAG) There is path")
    except:
        print("(DAG) There isn't path")

    print("\n" + "-" * 50 + "\n")

(Taiwan, XML)

(RAW) There is path
(DAG) There isn't path

--------------------------------------------------

(Stalinism, Socialism)

(RAW) There is path
(DAG) There isn't path

--------------------------------------------------

(Entertainers, 1940s in Japan)

(RAW) There is path
(DAG) There is path

--------------------------------------------------

(Politicians, French politicians)

(RAW) There is path
(DAG) There is path

--------------------------------------------------



## Features

What information extract from the infoboxes and categories for conditioning the generation of BART? 

How to represent the information? Graph structures? Raw text? 

How to combine the information?

### Infoboxes


Given:

* A tuple of entities E = {e1, ..., e_|E|}

* The predicate:object pairs from the infobox of each entity T = {t_1, ..., t_|T| : t_i = {(p_i1, o_i1), ...}

A graph G = (V, E) where V are objects and entities, and E = {(u, p, v) | u is an entity connected to the object v by means of a predicate p}

In [7]:
def f(graph, entities):
    
    def preprocess_predicate(pred):
        # Remove numbers for multiple predicates (office, office2, ...)
        pred = "".join(filter(lambda c: not c.isdigit(), pred))
        return pred
    
    def preprocess_object(obj):
        objs = re.findall("(\[\[.*?\]\])", obj)
        objs = [obj.split("|")[0].split("#")[0] for obj in objs]
        objs = [obj.replace("[", "").replace("]", "") for obj in objs]
        return objs
        
    def parse_infobox(infobox):
        

        def parse_coordinates(obj):
            """
            The parser fails with some coordinate formats, but, if clean
            different information can be extracted:
                · Coordinates
                · Region (ES, FR, GB, ..)
            """
            obj = obj.replace("}", "").replace("{", "").split("|")[1:]
            
            # region Coordinates #
            coordinates = None
            coord_delim = [obj.index(k) for k in ["E", "W"] if k in obj]
            coord_delim = coord_delim[0] if coord_delim else None
            
            if coord_delim is not None:
                coordinates = " ".join(obj[:coord_delim + 1]) # Specify degrees
            
            # endregion Coordinates #
            
            # region CoordinatesRegion #
            coord_region = None
            for k in obj:
                if "region" in k:
                    coord_region = k.split(":")[1]
                    break
            # endregion CoordinatesRegion #
            
            return (("coordinates", coordinates),
                    ("coordinates_region", coord_region))
            
        pred_obj_pairs = []
        for pred, obj in list(infobox.items()):
            
            # Treat manually-determined predicates (coordinates, ISIN for org, ...)
            if pred == "coordinates":
                coordinates_info = parse_coordinates(obj)
                for (pred, obj) in coordinates_info:
                    if obj is not None:
                        pred_obj_pairs.append((pred, obj))
            
            # Extract pairs where the object is a reference to a resource in Wikipedia
            if obj is not None and "[[" in obj:
                objs = preprocess_object(obj)
                pred = preprocess_predicate(pred)
                for obj in objs:
                    pred_obj_pairs.append((pred, obj))
                    
        return pred_obj_pairs     
            
    
    triples = []
    for entity in entities:
        infobox = wptools.page(entity).get_parse().data["infobox"]
        pred_obj_pairs = parse_infobox(infobox)
        triples.extend([(entity, obj, {"label": pred}) for (pred, obj) in pred_obj_pairs])
    
    graph.add_edges_from(triples)

In [8]:
info_graph = nx.DiGraph()
entities = ["David Stern", "Paul Tagliabue"]
f(info_graph, entities)

en.wikipedia.org (parse) David Stern
en.wikipedia.org (imageinfo) File:David Stern.jpg
David Stern (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:David Ste...
  infobox: <dict(19)> name, image, order, office, term_start, term...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 287571
  parsetree: <str(39822)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: David Stern
  wikibase: Q347958
  wikidata_url: https://www.wikidata.org/wiki/Q347958
  wikitext: <str(30624)> {{short description|American businessman,...
}
en.wikipedia.org (parse) Paul Tagliabue
en.wikipedia.org (imageinfo) File:Paul Tagliabue crop.jpg
Paul Tagliabue (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Paul Tagl...
  infobox: <dict(15)> name, image, caption, office, term_start, te...
  pageid: 272734
  parsetree: <str(22545)> <root><template><title>short description...
  requests: <list(2)> parse, image

In [9]:
g = net.Network(notebook=True, height="700px", width="1000px", directed=True)
g.show_buttons(filter_=['physics'])
g.from_nx(info_graph)
g.show("_.html")

### Categories

Given:

* A tuple of entities E = {e1, ..., e_|E|}
* The categories of the wikipedia page of each entity, C = {c_1, ..., c_|C| : c_i = {c_i1, ..., c_ij, ...}}
* A category graph G = (V, E) where V are the categories and E = {(u, v) | u!=v and v is a subcategory of u}

A new graph G' = (V', E') is created by adding E to V (V') and (c_ij, e_i) edges to E (E').


<img src="./graphfig.jpg" width=500 height=400>


Generalization (bottom-up traversal):

* Lowest common ancestors of E \in G'.


* Subgraph induced by the shortest paths between the ancestors and the entities. It's like a more general picture of the entities in G'.


* ...

Specialization (top-down traversal):

* Highest common descendants

* ...

In [5]:
def f(entities, site, dag):
    
    def clean_category_title(raw_title):
        for k in [":Category:", "]", "[", "en:"]:
            raw_title = raw_title.replace(k, "")
        return raw_title.split("|")[0]
    
    for entity in entities:
        page = list(site.search(entity, total=1, where="title"))
        if page:
            page = page[0]
            categories = list(page.categories())
            categories = [clean_category_title(cat.title(as_link=True,
                                                         textlink=True,
                                                         with_ns=False)) for cat in categories]
            
            # Add entity and edges (category, entity) if category exists in dag
            if not dag.has_node(entity):
                dag.add_node(entity)
            
                for category in categories:
                    if dag.has_node(category) and category not in entities:
                        dag.add_edge(category, entity)
    
    return dag

entities = ["Paris", "France"]
#entities = ["State Department", "Supreme Court"]
#entities = ["Ségolène Royal", "François Bayrou", "Nicolas Sarkozy"]
site = pywikibot.Site()
dag = f(entities, site, dag)

In [6]:
def lowest_common_ancestors(graph, nodes, top_k):
    #assert nx.is_directed_acyclic_graph(graph), "Graph has to be acyclic and directed."

    common_ancestors = list(set.intersection(*[nx.ancestors(graph, node) for node in nodes if node != "Living people" or node != "People"]))

    if not common_ancestors:
        return []
    
    sum_of_path_lengths = np.zeros((len(common_ancestors)))
    for ii, c in enumerate(common_ancestors):
        sum_of_path_lengths[ii] = functools.reduce(lambda x, y: x + y, 
                                                   [nx.shortest_path_length(graph, c, node) 
                                                         for node in nodes])

    indices = np.argsort(sum_of_path_lengths)[:top_k]   
    return [common_ancestors[ii] for ii in indices], sum_of_path_lengths[indices]

In [12]:
top_k = 10

lowest_ancestors = lowest_common_ancestors(dag, entities, top_k=top_k)
filtered_lower_ancestors = []
print("(%s, %s)\n" % (entities[0], entities[1]))
for ancestor, length in zip(*lowest_ancestors):
    print("Ancestor: %s, length: %d\n" % (ancestor, length))

(Paris, France)

Ancestor: G20 nations, length: 5

Ancestor: Southwestern European countries, length: 5

Ancestor: Member states of the Union for the Mediterranean, length: 5

Ancestor: Administrative territorial entities, length: 5

Ancestor: Member states of the Organisation internationale de la Francophonie, length: 5

Ancestor: Group of Eight nations, length: 5

Ancestor: Member states of the Council of Europe, length: 5

Ancestor: Republics, length: 5

Ancestor: Geography of Europe, length: 5

Ancestor: Use British English from April 2015, length: 5



In [76]:
print("Subgraph induced by the shortest paths among ancestors and entities")
print("Virginia and Vermont are States of the United States but Vermont is a New England state and Virginia not")
induced_vertices = []
for ancestor in lowest_ancestors[0]:
    for entity in entities:
        induced_vertices.extend(nx.shortest_path(dag, ancestor, entity))

g = net.Network(notebook=True, height="500px", width="900px", directed=True)
g.show_buttons(filter_=['physics'])
g.from_nx(dag.subgraph(induced_vertices))
g.show("_.html")

Subgraph induced by the shortest paths among ancestors and entities
Virginia and Vermont are States of the United States but Vermont is a New England state and Virginia not


### Combining the information


Combine both infobox and category in one graph, labeling the categories edges with a predicate "category".


In [ ]:


# Guardar edgelist para lanzar Breaking Cycles in Noisy Hierarchies
labels = {}
c = 0
graph = graph.reverse()
for node in graph.nodes():
    if node not in labels:
        labels[node] = c
        c += 1

with open("relabeling_indices.pkl", "wb") as fw:
    pkl.dump( {v: k for k, v in labels.items()}, fw)
int_graph = nx.relabel_nodes(graph, labels)
nx.write_edgelist(int_graph, "tesa.edges",data = False)

with open("relabeling_indices.pkl", "rb") as fr:
    inverse_labeling = pkl.load(fr)

# Conditioning the generation


Use the common ancestors of E as hierarchical labeling: 

  * A Hierarchical Neural Attention-based Text Classifier: https://www.aclweb.org/anthology/D18-1094.pdf
    
  * Hierarchical Multi-Label Classification Networks: http://proceedings.mlr.press/v80/wehrmann18a/wehrmann18a.pdf

